# Load modules

In [1]:
# Import necessary modules
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Functions for parsing, formatting and encoding data

In [2]:
def parse_pssm(pssm_filename):
  amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
  num_aas = len(amino_acids)
  sequence = ''
  profile = []

  # Parsing MSA frequences from a PSSM file 
  with open(pssm_filename) as pssm:
    pssm_lines = pssm.readlines()
    for line in pssm_lines[3:-6]:                 # First iterates over lines
      profile_line = []
      for n in line.rstrip().split()[22:-2]:      # Then over n in line 
        profile_line.append(float(n)/100)         # Converts the values to a scale of 0 to 1
      profile.append(profile_line)
      sequence += line[6]                         # Fetches the protein sequence: every 6th character in given line n


  # One-hot encoding the protein sequence
  encoding = np.zeros((len(sequence), num_aas))   # Initialises a 2D array of zeros
  for i, aa in enumerate(sequence):               # Returns an iterator that produces tuples containing both the index and aa
    if aa in amino_acids:
      index = amino_acids.index(aa)               # Finds corresponding index at aa string 
      encoding[i, index] = 1                      # 0 is replaced with 1 in the array, at position: seq index x aa string index
    else: encoding[i, :] = 0.05                   # If aa not found in file, fill the entire row with 0.05 to represent unknown/invalid aa

  return encoding, profile


# Parsing the DSSP labels
def parse_dssp(dssp_filename):
	ss = ''
	with open(dssp_filename) as dssp:
		dssp.readline()
		ss = dssp.readline().rstrip()
	return ss

# Function for fetching appropriate data

In [3]:
def fetch_data(window, cv=False, train=True):
  ss_map = {'H':[1,0,0], 'E':[0,1,0], 'C':[0,0,1], '-':[0,0,1]}

  # Make sure that window is an odd integer
  assert type(window) == int, 'Error: window must be an integer!'
  assert window%2!=0, 'Error: window must be an ODD integer!'


  # Selecting which data to fetch (training or blind test) and loading the right list of ids
  if not cv:
    path = 'C:/Users/liisa/OneDrive/Dokumendid/ROOTSI/SU/Bioinformatics/Project/protein-ss-pred-master/data/blindTest/'
    id_list = [line.rstrip() for line in open(path+'list.txt')]
    print ('Test data is obtained:')
  else:
    cv = str(cv)
    path = 'C:/Users/liisa/OneDrive/Dokumendid/ROOTSI/SU/Bioinformatics/Project/protein-ss-pred-master/data/training/'
    if train:
      id_list = [line.rstrip() for line in open(path+'/cv/train'+cv+'.txt')]           # Using list comprehension to store train set id-s
      print ('Train partition of cross-validation set {} is obtained:'.format(cv))
    else:
      id_list = [line.rstrip() for line in open(path+'/cv/test'+cv+'.txt')]
      print ('Test partition of cross-validation set {} is obtained:'.format(cv))

  X, Y = [], []
  for id in id_list:
    # Fetching the input features, using list comprehension to store in list x
    id = id.replace(":", "_")
    sequence, profile = np.array(parse_pssm(path+'/pssm/'+id+'.pssm'))                 # sequence = one-hot encoded sequence
    x = np.concatenate((sequence, profile), axis=-1)                        
 
    # Fetching, encoding labels, storing in array y
    ss = parse_dssp(path+'/dssp/'+id+'.dssp')
    y = np.array([ss_map[c] for c in ss])

    # Adding (window-1)/2 padding on both sequence sides
    # Creating windows for first and last positions
    side = int((window-1)/2)
    x_pad = np.zeros((side, 40))
    x = np.concatenate((x_pad, x, x_pad), axis=0)

    # Extracting all windows:
    # For each index i in the range from side to len(x)-side-1, it creates a window centered at position i
    # Each window is extracted as a subarray of x from index i-side to i+side+1
    X += [x[i-side:i+side+1,:] for i in range(side, len(x)-side-1)]                   
    Y += [y[i,:] for i in range(len(y))]                                              # Corresponding labels for Y
    X.append(x[-2*side-1:,:])                                                         # Extracts a window of size 2*side+1 from beginning of sequence and appends to X

  return np.array(X), np.array(Y)

# Declare and train a neural network

In [4]:
window = 17
input_shape = (window, 40)
num_classes = 3

model = keras.Sequential([
        keras.Input(shape=input_shape),
        keras.layers.Flatten(),
        keras.layers.Dense(256, activation="leaky_relu"),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(128, activation="leaky_relu"),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(32, activation="relu"),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(num_classes, activation='softmax')
    ])

model.summary() 
model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 680)               0         
                                                                 
 dense (Dense)               (None, 256)               174336    
                                                                 
 batch_normalization (Batch  (None, 256)               1024      
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 batch_normalization_1 (Bat  (None, 128)               512       
 chNormalization)                                      

# Fetch data

In [5]:
models = []
for fold in range(1, 6):
    # Loading training and testing data for the current fold
    x_train, y_train = fetch_data(window, cv=fold, train=True)
    print(x_train.shape, y_train.shape)
    x_test, y_test = fetch_data(window, cv=fold, train=False)
    print(x_test.shape, y_test.shape)

    # Training model on the training data
    history = model.fit(x_train, y_train, epochs=10, batch_size=512, validation_split=0.1)
    
    # Evaluating model on the testing data
    loss, accuracy = model.evaluate(x_test, y_test)
    models.append(model)
    
    
    print("Fold {0}: Loss = {1}, Accuracy = {2}".format(fold, round(loss,2), round(accuracy,2)))

Train partition of cross-validation set 1 is obtained:
(157415, 17, 40) (157415, 3)
Test partition of cross-validation set 1 is obtained:
(40729, 17, 40) (40729, 3)
Epoch 1/10


277/277 [==============================] - 4s 9ms/step - loss: 0.9518 - accuracy: 0.5743 - val_loss: 0.7157 - val_accuracy: 0.7080
Epoch 2/10
277/277 [==============================] - 2s 8ms/step - loss: 0.7530 - accuracy: 0.6861 - val_loss: 0.6655 - val_accuracy: 0.7273
Epoch 3/10
277/277 [==============================] - 2s 8ms/step - loss: 0.7243 - accuracy: 0.7015 - val_loss: 0.6560 - val_accuracy: 0.7318
Epoch 4/10
277/277 [==============================] - 2s 8ms/step - loss: 0.7006 - accuracy: 0.7118 - val_loss: 0.6343 - val_accuracy: 0.7423
Epoch 5/10
277/277 [==============================] - 2s 8ms/step - loss: 0.6757 - accuracy: 0.7237 - val_loss: 0.6132 - val_accuracy: 0.7493
Epoch 6/10
277/277 [==============================] - 2s 9ms/step - loss: 0.6505 - accuracy: 0.7346 - val_loss: 0.6012 - va

# Fetch blind test data, use fitted model to get predictions, and evaluate predictions

In [6]:
x_blind_test, y_blind_test = fetch_data(window, cv=False, train=False)
print(x_blind_test.shape, y_blind_test.shape)

Test data is obtained:
(62136, 17, 40) (62136, 3)


In [7]:
trained_model = models[4]
trained_model.save('C:/Users/liisa/OneDrive/Dokumendid/ROOTSI/SU/Bioinformatics/Project/trained_FCNN.keras')
predictions = trained_model.predict(x_blind_test)

# Reporting model's accuracy
print("Accuracy: ", round(accuracy_score(np.argmax(y_blind_test, axis=1), np.argmax(predictions, axis=1)),2))

# Converting one-hot encoded labels back to categorical labels
y_blind_test_cat = np.argmax(y_blind_test, axis=1)
predictions_cat = np.argmax(predictions, axis=1)

# Calculating F1 score for each class
F1_scores = f1_score(y_blind_test_cat, predictions_cat, average=None)

ss_labels = ['H', 'E', 'C']
# Printing F1 scores for each class
for label, score in zip(ss_labels, F1_scores):
    print("Secondary Structure Class: {0}, F1-score: {1}".format(label,round(score,2)))

1942/1942 [==============================] - 2s 984us/step
Accuracy:  0.75
Secondary Structure Class: H, F1-score: 0.79
Secondary Structure Class: E, F1-score: 0.7
Secondary Structure Class: C, F1-score: 0.74
